In [24]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [32]:
import numpy as np
from sweep.experiment import Experiment, ExperimentType
from model.data import DataModel, KFeaturesDefinition, k_features_factory
from erm.problems.problems import ProblemType
import subprocess
from experiments.data_loading import read_result_dataframe

Define a datamodel

In [26]:
d = 100
vanilla_gaussian_diagonal = [(1, d)]
x_diagonal = KFeaturesDefinition(diagonal=vanilla_gaussian_diagonal)
θ_diagonal = KFeaturesDefinition(diagonal=vanilla_gaussian_diagonal)
ω_diagonal = KFeaturesDefinition(diagonal=vanilla_gaussian_diagonal)
δ_diagonal = KFeaturesDefinition(diagonal=vanilla_gaussian_diagonal)
ν_diagonal = KFeaturesDefinition(diagonal=vanilla_gaussian_diagonal)

k_features_kwargs = {
    "x_diagonal": x_diagonal,
    "θ_diagonal": θ_diagonal,
    "ω_diagonal": ω_diagonal,
    "δ_diagonal": δ_diagonal,
    "ν_diagonal": ν_diagonal,
}

vanilla_gaussian_datamodel = DataModel(
    d,
    normalize_matrices=False,
    data_model_factory=k_features_factory,
    factory_kwargs=k_features_kwargs,
    name="vanilla-gaussian"
)


In [27]:
two_features_diagonal = [(1, int(d/2)), (2, int(d/2))]
x_diagonal = KFeaturesDefinition(diagonal=two_features_diagonal)
θ_diagonal = KFeaturesDefinition(diagonal=two_features_diagonal)
ω_diagonal = KFeaturesDefinition(diagonal=two_features_diagonal)
δ_diagonal = KFeaturesDefinition(diagonal=two_features_diagonal)
ν_diagonal = KFeaturesDefinition(diagonal=two_features_diagonal)

k_features_kwargs = {
    "x_diagonal": x_diagonal,
    "θ_diagonal": θ_diagonal,
    "ω_diagonal": ω_diagonal,
    "δ_diagonal": δ_diagonal,
    "ν_diagonal": ν_diagonal,
}
two_features_model = DataModel(
    d,
    normalize_matrices=False,
    data_model_factory=k_features_factory,
    factory_kwargs=k_features_kwargs,
    name="two-features-model"
)

Define an experiment

In [28]:
experiment = Experiment(
    state_evolution_repetitions=1,
    erm_repetitions=1,
    alphas=np.linspace(0.01,10,5),
    epsilons=np.array([0, 0.1]),
    lambdas=np.array([0.1]),
    taus=np.array([0.1]),
    d=d,
    experiment_type=ExperimentType.Sweep,
    data_models = [vanilla_gaussian_datamodel, two_features_model],
    test_against_epsilons=np.array([0,0.1]),
    erm_problem_type=ProblemType.Logistic,
    gamma_fair_error=0.01,
    name="vanilla-gaussian"
)
experiment_json = experiment.to_json()

In [29]:
with open("experiment.json", "w") as f:
    f.write(experiment_json)

In [30]:
command = ["pwd"]
subprocess.run(command)

/Users/kasimirtanner/Documents/LocalCode/Code_PDM/AISTATS


CompletedProcess(args=['pwd'], returncode=0)

In [31]:
venv_python = ".venv/bin/python"
command = ["mpiexec", "-n", "5", venv_python, "sweep/run_sweep.py", "--json", experiment_json, "--log-level", "INFO"]
subprocess.run(command)

2025-02-17 21:40:05,559 - Rank 0 - INFO - The MPI comm size is 5
2025-02-17 21:40:05,559 - Rank 0 - INFO - This process has rank 0
2025-02-17 21:40:05,559 - Rank 0 - INFO - Starting Master for experiment vanilla-gaussian
2025-02-17 21:40:05,559 - Rank 0 - INFO - Starting Experiment vanilla-gaussian
2025-02-17 21:40:05,571 - Rank 0 - INFO - Starting all processes
2025-02-17 21:40:05,571 - Rank 0 - INFO - Sending task 0 to 1
2025-02-17 21:40:05,572 - Rank 0 - INFO - Sending task 1 to 2
2025-02-17 21:40:05,572 - Rank 0 - INFO - Sending task 2 to 3
2025-02-17 21:40:05,572 - Rank 0 - INFO - Sending task 3 to 4
2025-02-17 21:40:05,572 - Rank 0 - INFO - All processes started - receiving results and sending new tasks
2025-02-17 21:40:05,572 - Rank 0 - INFO - Received the 0th task


INFO:root:The MPI comm size is 5
INFO:root:This process has rank 0
INFO:root:Starting Master for experiment vanilla-gaussian
INFO:root:Starting Experiment vanilla-gaussian
INFO:root:The MPI comm size is 5
INFO:root:The MPI comm size is 5
INFO:root:The MPI comm size is 5
INFO:root:This process has rank 3
INFO:root:The MPI comm size is 5
INFO:root:This process has rank 2
INFO:root:This process has rank 4
INFO:root:This process has rank 1
  0%|          | 0/40 [00:00<?, ?it/s]INFO:root:Starting all processes
INFO:root:Sending task 0 to 1
INFO:root:Sending task 1 to 2
INFO:root:Sending task 2 to 3
INFO:root:Sending task 3 to 4
INFO:root:All processes started - receiving results and sending new tasks
INFO:root:Received the 0th task
INFO:root:Starting task 2
INFO:root:Starting task 3
INFO:root:Initializing data model
INFO:root:Starting task 1
INFO:root:Initializing data model
INFO:root:Initializing data model
INFO:root:Starting task 4
INFO:root:Initializing data model
INFO:root:d: 100
INFO:r

2025-02-17 21:40:07,957 - Rank 0 - INFO - Received task 2 from 2
2025-02-17 21:40:07,957 - Rank 0 - INFO - Saved Task(id=2, task_type=<TaskType.SE: 1>, erm_problem_type=None, alpha=0.01, epsilon=0.0, test_against_epsilons=[0.0, 0.1], lam=0.1, tau=0.1, d=100, data_model=DataModel(d=100, normalize_matrices=False, data_model_factory=k_features_factory, factory_kwargs={'x_diagonal': KFeaturesDefinition(diagonal=[[1, 50], [2, 50]]), 'θ_diagonal': KFeaturesDefinition(diagonal=[[1, 50], [2, 50]]), 'ω_diagonal': KFeaturesDefinition(diagonal=[[1, 50], [2, 50]]), 'δ_diagonal': KFeaturesDefinition(diagonal=[[1, 50], [2, 50]]), 'ν_diagonal': KFeaturesDefinition(diagonal=[[1, 50], [2, 50]])}, name='two-features-model', description='', gamma=1), values={}, gamma_fair_error=0.01, gamma=1, result=<sweep.results.state_evolution.SEResult object at 0x154eaed90>)
2025-02-17 21:40:07,957 - Rank 0 - INFO - Received the 1th task
2025-02-17 21:40:07,958 - Rank 0 - INFO - Received task 4 from 4
2025-02-17 21:4

INFO:root:d: 100
INFO:root:normalize_matrices: False
INFO:root:Ratio of first to last eigenvalue of Σ_x: 1.0
INFO:root:Norm Σ_x: 100.0
INFO:root:Norm Σ_ω: 100.0
INFO:root:Norm Σ_δ: 100.0
INFO:root:Norm Σ_ν: 100.0
INFO:root:Σ_x: [[1. 0. 0. ... 0. 0. 0.]
 [0. 1. 0. ... 0. 0. 0.]
 [0. 0. 1. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 1. 0. 0.]
 [0. 0. 0. ... 0. 1. 0.]
 [0. 0. 0. ... 0. 0. 1.]]
INFO:root:Σ_ω: [[1. 0. 0. ... 0. 0. 0.]
 [0. 1. 0. ... 0. 0. 0.]
 [0. 0. 1. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 1. 0. 0.]
 [0. 0. 0. ... 0. 1. 0.]
 [0. 0. 0. ... 0. 0. 1.]]
INFO:root:Σ_δ: [[1. 0. 0. ... 0. 0. 0.]
 [0. 1. 0. ... 0. 0. 0.]
 [0. 0. 1. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 1. 0. 0.]
 [0. 0. 0. ... 0. 1. 0.]
 [0. 0. 0. ... 0. 0. 1.]]
INFO:root:Σ_ν: [[1. 0. 0. ... 0. 0. 0.]
 [0. 1. 0. ... 0. 0. 0.]
 [0. 0. 1. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 1. 0. 0.]
 [0. 0. 0. ... 0. 1. 0.]
 [0. 0. 0. ... 0. 0. 1.]]
INFO:root:Σ_x value counts: (array([0., 1.]), array([9900,  100]))
INFO:root:Σ_ω value counts: (array([

2025-02-17 21:40:09,269 - Rank 0 - INFO - Received task 5 from 2
2025-02-17 21:40:09,269 - Rank 0 - INFO - Saved Task(id=5, task_type=<TaskType.SE: 1>, erm_problem_type=None, alpha=2.5075, epsilon=0.0, test_against_epsilons=[0.0, 0.1], lam=0.1, tau=0.1, d=100, data_model=DataModel(d=100, normalize_matrices=False, data_model_factory=k_features_factory, factory_kwargs={'x_diagonal': KFeaturesDefinition(diagonal=[[1, 100]]), 'θ_diagonal': KFeaturesDefinition(diagonal=[[1, 100]]), 'ω_diagonal': KFeaturesDefinition(diagonal=[[1, 100]]), 'δ_diagonal': KFeaturesDefinition(diagonal=[[1, 100]]), 'ν_diagonal': KFeaturesDefinition(diagonal=[[1, 100]])}, name='vanilla-gaussian', description='', gamma=1), values={}, gamma_fair_error=0.01, gamma=1, result=<sweep.results.state_evolution.SEResult object at 0x154eb4550>)
2025-02-17 21:40:09,270 - Rank 0 - INFO - Received the 5th task
2025-02-17 21:40:09,272 - Rank 0 - INFO - Received task 6 from 4
2025-02-17 21:40:09,272 - Rank 0 - INFO - Saved Task(id

...

INFO:root:Finished ERM Task(id=39, task_type=<TaskType.ERM: 0>, erm_problem_type=<ProblemType.Logistic: 0>, alpha=10.0, epsilon=0.1, test_against_epsilons=[0.0, 0.1], lam=0.1, tau=0.1, d=100, data_model=DataModel(d=100, normalize_matrices=False, data_model_factory=k_features_factory, factory_kwargs={'x_diagonal': KFeaturesDefinition(diagonal=[[1, 100]]), 'θ_diagonal': KFeaturesDefinition(diagonal=[[1, 100]]), 'ω_diagonal': KFeaturesDefinition(diagonal=[[1, 100]]), 'δ_diagonal': KFeaturesDefinition(diagonal=[[1, 100]]), 'ν_diagonal': KFeaturesDefinition(diagonal=[[1, 100]])}, name='vanilla-gaussian', description='', gamma=1), values={}, gamma_fair_error=0.01, gamma=1, result=None)
INFO:root:Received task 39 from 3
INFO:root:Saved Task(id=39, task_type=<TaskType.ERM: 0>, erm_problem_type=<ProblemType.Logistic: 0>, alpha=10.0, epsilon=0.1, test_against_epsilons=[0.0, 0.1], lam=0.1, tau=0.1, d=100, data_model=DataModel(d=100, normalize_matrices=False, data_model_factory=k_features_factory,

2025-02-17 21:40:33,543 - Rank 0 - INFO - Received task 39 from 3
2025-02-17 21:40:33,543 - Rank 0 - INFO - Saved Task(id=39, task_type=<TaskType.ERM: 0>, erm_problem_type=<ProblemType.Logistic: 0>, alpha=10.0, epsilon=0.1, test_against_epsilons=[0.0, 0.1], lam=0.1, tau=0.1, d=100, data_model=DataModel(d=100, normalize_matrices=False, data_model_factory=k_features_factory, factory_kwargs={'x_diagonal': KFeaturesDefinition(diagonal=[[1, 100]]), 'θ_diagonal': KFeaturesDefinition(diagonal=[[1, 100]]), 'ω_diagonal': KFeaturesDefinition(diagonal=[[1, 100]]), 'δ_diagonal': KFeaturesDefinition(diagonal=[[1, 100]]), 'ν_diagonal': KFeaturesDefinition(diagonal=[[1, 100]])}, name='vanilla-gaussian', description='', gamma=1), values={}, gamma_fair_error=0.01, gamma=1, result=<sweep.results.erm.ERMResult object at 0x154ee5a90>)
2025-02-17 21:40:33,543 - Rank 0 - INFO - Received the 38th task
2025-02-17 21:40:33,599 - Rank 0 - INFO - Received task 38 from 2
2025-02-17 21:40:33,599 - Rank 0 - INFO - 

INFO:root:Finished ERM Task(id=40, task_type=<TaskType.ERM: 0>, erm_problem_type=<ProblemType.Logistic: 0>, alpha=10.0, epsilon=0.1, test_against_epsilons=[0.0, 0.1], lam=0.1, tau=0.1, d=100, data_model=DataModel(d=100, normalize_matrices=False, data_model_factory=k_features_factory, factory_kwargs={'x_diagonal': KFeaturesDefinition(diagonal=[[1, 50], [2, 50]]), 'θ_diagonal': KFeaturesDefinition(diagonal=[[1, 50], [2, 50]]), 'ω_diagonal': KFeaturesDefinition(diagonal=[[1, 50], [2, 50]]), 'δ_diagonal': KFeaturesDefinition(diagonal=[[1, 50], [2, 50]]), 'ν_diagonal': KFeaturesDefinition(diagonal=[[1, 50], [2, 50]])}, name='two-features-model', description='', gamma=1), values={}, gamma_fair_error=0.01, gamma=1, result=None)
INFO:root:Received task 40 from 1
INFO:root:Saved Task(id=40, task_type=<TaskType.ERM: 0>, erm_problem_type=<ProblemType.Logistic: 0>, alpha=10.0, epsilon=0.1, test_against_epsilons=[0.0, 0.1], lam=0.1, tau=0.1, d=100, data_model=DataModel(d=100, normalize_matrices=Fal

2025-02-17 21:40:34,269 - Rank 0 - INFO - Received task 40 from 1
2025-02-17 21:40:34,275 - Rank 0 - INFO - Saved Task(id=40, task_type=<TaskType.ERM: 0>, erm_problem_type=<ProblemType.Logistic: 0>, alpha=10.0, epsilon=0.1, test_against_epsilons=[0.0, 0.1], lam=0.1, tau=0.1, d=100, data_model=DataModel(d=100, normalize_matrices=False, data_model_factory=k_features_factory, factory_kwargs={'x_diagonal': KFeaturesDefinition(diagonal=[[1, 50], [2, 50]]), 'θ_diagonal': KFeaturesDefinition(diagonal=[[1, 50], [2, 50]]), 'ω_diagonal': KFeaturesDefinition(diagonal=[[1, 50], [2, 50]]), 'δ_diagonal': KFeaturesDefinition(diagonal=[[1, 50], [2, 50]]), 'ν_diagonal': KFeaturesDefinition(diagonal=[[1, 50], [2, 50]])}, name='two-features-model', description='', gamma=1), values={}, gamma_fair_error=0.01, gamma=1, result=<sweep.results.erm.ERMResult object at 0x154eb6890>)
2025-02-17 21:40:34,275 - Rank 0 - INFO - All tasks sent and received
2025-02-17 21:40:34,290 - Rank 0 - INFO - Marking experiment 

CompletedProcess(args=['mpiexec', '-n', '5', '.venv/bin/python', 'sweep/run_sweep.py', '--json', '{"name": "vanilla-gaussian", "state_evolution_repetitions": 1, "erm_repetitions": 1, "alphas": [0.01, 2.5075, 5.005, 7.5024999999999995, 10.0], "epsilons": [0.0, 0.1], "test_against_epsilons": [0.0, 0.1], "lambdas": [0.1], "taus": [0.1], "d": 100, "experiment_type": "Sweep", "data_models": [{"d": 100, "normalize_matrices": false, "data_model_factory_name": "k_features_factory", "factory_kwargs": {"x_diagonal": {"diagonal": [[1, 100]]}, "\\u03b8_diagonal": {"diagonal": [[1, 100]]}, "\\u03c9_diagonal": {"diagonal": [[1, 100]]}, "\\u03b4_diagonal": {"diagonal": [[1, 100]]}, "\\u03bd_diagonal": {"diagonal": [[1, 100]]}}, "name": "vanilla-gaussian", "description": "", "gamma": 1}, {"d": 100, "normalize_matrices": false, "data_model_factory_name": "k_features_factory", "factory_kwargs": {"x_diagonal": {"diagonal": [[1, 50], [2, 50]]}, "\\u03b8_diagonal": {"diagonal": [[1, 50], [2, 50]]}, "\\u03c

In [33]:
df_erm, df_se = read_result_dataframe(experiment.name)